# **Adversarial Training and Robustness of VGG-16, ResNet18, and EfficientNet-B0 on CIFAR-10**
# **Pretrain=False, 10 epoch**

## **1. Import Required Libraries**

In [1]:

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchattacks
import time
import pandas as pd
from torchvision import models

# Check for GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')


Using device: cuda


## **2. Load CIFAR-10 Dataset**

In [2]:

# Load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize for consistency across models
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False)


# **3. Defining Models From torchvision**

## **3.1 Define VGG-16 Model**

In [3]:

# Load pre-defined VGG-16 model from torchvision
class VGG16(nn.Module):
    def __init__(self):
        super(VGG16, self).__init__()
        self.model = models.vgg16(pretrained=False)
        num_ftrs = self.model.classifier[6].in_features
        self.model.classifier[6] = nn.Linear(num_ftrs, 10)  # Adjust for CIFAR-10 classes

    def forward(self, x):
        return self.model(x)

# Initialize model
model_vgg = VGG16().to(device)
criterion = nn.CrossEntropyLoss()
optimizer_vgg = optim.Adam(model_vgg.parameters(), lr=0.001)


c:\Users\cheey\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\cheey\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


## **3.2 Define ResNet18 Model**

In [4]:

# Load pre-defined ResNet18 model from torchvision
class ResNet18(nn.Module):
    def __init__(self):
        super(ResNet18, self).__init__()
        self.model = models.resnet18(pretrained=False)
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, 10)  # Adjust for CIFAR-10 classes

    def forward(self, x):
        return self.model(x)

# Initialize model
model_resnet = ResNet18().to(device)
optimizer_resnet = optim.Adam(model_resnet.parameters(), lr=0.001)


## **3.3 Define EfficientNet-B0 Model**

In [5]:

# Load pre-defined EfficientNet-B0 model from torchvision
class EfficientNetB0(nn.Module):
    def __init__(self):
        super(EfficientNetB0, self).__init__()
        self.model = models.efficientnet_b0(pretrained=False)
        num_ftrs = self.model.classifier[1].in_features
        self.model.classifier[1] = nn.Linear(num_ftrs, 10)  # Adjust for CIFAR-10 classes

    def forward(self, x):
        return self.model(x)

# Initialize model
model_efficient = EfficientNetB0().to(device)
optimizer_efficient = optim.Adam(model_efficient.parameters(), lr=0.001)


## **4. Train the Models with Adversarial Training (FGSM)**

In [6]:
import gc
torch.cuda.empty_cache()
gc.collect()


0

In [7]:

# FGSM attack for adversarial training
def adversarial_training(model, optimizer, model_name, epochs=10):
    fgsm_attack = torchattacks.FGSM(model, eps=0.03)
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for images, labels in trainloader:
            images, labels = images.to(device), labels.to(device)

            # Generate adversarial examples using FGSM
            adv_images = fgsm_attack(images, labels)

            optimizer.zero_grad()
            outputs = model(adv_images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

        print(f"{model_name} - Epoch {epoch+1}, Loss: {running_loss/len(trainloader):.4f}")

print("Training VGG-16 with FGSM adversarial training...")
adversarial_training(model_vgg, optimizer_vgg, "VGG-16")

print("Training ResNet18 with FGSM adversarial training...")
adversarial_training(model_resnet, optimizer_resnet, "ResNet18")

print("Training EfficientNet-B0 with FGSM adversarial training...")
adversarial_training(model_efficient, optimizer_efficient, "EfficientNet-B0")


Training VGG-16 with FGSM adversarial training...
VGG-16 - Epoch 1, Loss: 2.4399
VGG-16 - Epoch 2, Loss: 2.3028
VGG-16 - Epoch 3, Loss: 2.3028
VGG-16 - Epoch 4, Loss: 2.3028
VGG-16 - Epoch 5, Loss: 2.3028
VGG-16 - Epoch 6, Loss: 2.3028
VGG-16 - Epoch 7, Loss: 2.3027
VGG-16 - Epoch 8, Loss: 2.3027
VGG-16 - Epoch 9, Loss: 2.3027
VGG-16 - Epoch 10, Loss: 2.3028
Training ResNet18 with FGSM adversarial training...
ResNet18 - Epoch 1, Loss: 1.7130
ResNet18 - Epoch 2, Loss: 1.4684
ResNet18 - Epoch 3, Loss: 1.2083
ResNet18 - Epoch 4, Loss: 1.0092
ResNet18 - Epoch 5, Loss: 0.8879
ResNet18 - Epoch 6, Loss: 0.7052
ResNet18 - Epoch 7, Loss: 0.6489
ResNet18 - Epoch 8, Loss: 0.5771
ResNet18 - Epoch 9, Loss: 0.4361
ResNet18 - Epoch 10, Loss: 0.3748
Training EfficientNet-B0 with FGSM adversarial training...
EfficientNet-B0 - Epoch 1, Loss: 1.8485
EfficientNet-B0 - Epoch 2, Loss: 1.3872
EfficientNet-B0 - Epoch 3, Loss: 1.3185
EfficientNet-B0 - Epoch 4, Loss: 1.1860
EfficientNet-B0 - Epoch 5, Loss: 1.06

## **5. Evaluate Models on Clean Test Images**

In [8]:

# Evaluation function
def test_model(model, dataloader, model_name):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'{model_name} Accuracy on clean test images: {accuracy:.2f}%')
    return accuracy

clean_accuracy_vgg = test_model(model_vgg, testloader, "VGG-16")
clean_accuracy_resnet = test_model(model_resnet, testloader, "ResNet18")
clean_accuracy_effnet = test_model(model_efficient, testloader, "EfficientNet-B0")


VGG-16 Accuracy on clean test images: 10.00%
ResNet18 Accuracy on clean test images: 15.56%
EfficientNet-B0 Accuracy on clean test images: 20.55%


# **6. Setting up for evaluating Adversarial attacks**

In [10]:
# Function to evaluate adversarial attacks
def test_adversarial_attack(model, attack, attack_name):
    model.eval()
    correct = 0
    total = 0
    success_count = 0
    start_time = time.time()

    for images, labels in testloader:
        images, labels = images.to(device), labels.to(device)
        adv_images = attack(images, labels)
        outputs = model(adv_images)
        _, predicted = torch.max(outputs.data, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        success_count += (predicted != labels).sum().item()

    end_time = time.time()
    attack_time = end_time - start_time
    attack_accuracy = 100 * correct / total
    attack_success_rate = 100 * success_count / total

    print(f'{attack_name} Attack Results:')
    print(f' - Accuracy after attack: {attack_accuracy:.2f}%')
    print(f' - Attack Success Rate: {attack_success_rate:.2f}%')
    print(f' - Time Taken: {attack_time:.2f} seconds\n')

    return attack_accuracy, attack_success_rate, attack_time

# **6.1 Evaluating Adversarial Robustness of VGG-16 with Adversarial Training (FGSM)**

In [9]:
# Define adversarial attacks
attacks = {
    "FGSM": torchattacks.FGSM(model_vgg, eps=0.03),
    "PGD": torchattacks.PGD(model_vgg, eps=0.03, alpha=0.01, steps=40),
    "DeepFool": torchattacks.DeepFool(model_vgg, steps=50),
    "BIM": torchattacks.BIM(model_vgg, eps=0.03, alpha=0.01, steps=10),
    "C&W": torchattacks.CW(model_vgg, c=1, kappa=0, steps=100, lr=0.01)
}

In [11]:
# Run attacks and collect results
attack_results = {}

for attack_name, attack in attacks.items():
    attack_results[attack_name] = test_adversarial_attack(model_vgg, attack, attack_name)

# Print summary
print("\nFinal Adversarial Attack Summary on VGG-16:")
for attack, (acc, success_rate, time_taken) in attack_results.items():
    print(f"{attack}:")
    print(f" - Accuracy After Attack: {acc:.2f}%")
    print(f" - Attack Success Rate: {success_rate:.2f}%")
    print(f" - Time Taken: {time_taken:.2f} sec\n")

FGSM Attack Results:
 - Accuracy after attack: 10.00%
 - Attack Success Rate: 90.00%
 - Time Taken: 2092.51 seconds

PGD Attack Results:
 - Accuracy after attack: 10.00%
 - Attack Success Rate: 90.00%
 - Time Taken: 62314.70 seconds

DeepFool Attack Results:
 - Accuracy after attack: 0.00%
 - Attack Success Rate: 100.00%
 - Time Taken: 2639.37 seconds

BIM Attack Results:
 - Accuracy after attack: 10.00%
 - Attack Success Rate: 90.00%
 - Time Taken: 18651.76 seconds

C&W Attack Results:
 - Accuracy after attack: 10.00%
 - Attack Success Rate: 90.00%
 - Time Taken: 197441.46 seconds


Final Adversarial Attack Summary on VGG-16:
FGSM:
 - Accuracy After Attack: 10.00%
 - Attack Success Rate: 90.00%
 - Time Taken: 2092.51 sec

PGD:
 - Accuracy After Attack: 10.00%
 - Attack Success Rate: 90.00%
 - Time Taken: 62314.70 sec

DeepFool:
 - Accuracy After Attack: 0.00%
 - Attack Success Rate: 100.00%
 - Time Taken: 2639.37 sec

BIM:
 - Accuracy After Attack: 10.00%
 - Attack Success Rate: 90.00

# **6.2 Evaluating Adversarial Robustness of ResNet18 with Adversarial Training (FGSM)**

In [12]:
# Define adversarial attacks
attacks = {
    "FGSM": torchattacks.FGSM(model_resnet, eps=0.03),
    "PGD": torchattacks.PGD(model_resnet, eps=0.03, alpha=0.01, steps=40),
    "DeepFool": torchattacks.DeepFool(model_resnet, steps=50),
    "BIM": torchattacks.BIM(model_resnet, eps=0.03, alpha=0.01, steps=10),
    "C&W": torchattacks.CW(model_resnet, c=1, kappa=0, steps=100, lr=0.01)
}

In [13]:
# Run attacks and collect results
attack_results = {}

for attack_name, attack in attacks.items():
    attack_results[attack_name] = test_adversarial_attack(model_resnet, attack, attack_name)

# Print summary
print("\nFinal Adversarial Attack Summary on ResNet18:")
for attack, (acc, success_rate, time_taken) in attack_results.items():
    print(f"{attack}:")
    print(f" - Accuracy After Attack: {acc:.2f}%")
    print(f" - Attack Success Rate: {success_rate:.2f}%")
    print(f" - Time Taken: {time_taken:.2f} sec\n")

FGSM Attack Results:
 - Accuracy after attack: 74.46%
 - Attack Success Rate: 25.54%
 - Time Taken: 232.66 seconds

PGD Attack Results:
 - Accuracy after attack: 6.46%
 - Attack Success Rate: 93.54%
 - Time Taken: 3336.66 seconds

DeepFool Attack Results:
 - Accuracy after attack: 6.04%
 - Attack Success Rate: 93.96%
 - Time Taken: 43134.19 seconds

BIM Attack Results:
 - Accuracy after attack: 7.89%
 - Attack Success Rate: 92.11%
 - Time Taken: 1779.63 seconds

C&W Attack Results:
 - Accuracy after attack: 3.75%
 - Attack Success Rate: 96.25%
 - Time Taken: 9772.34 seconds


Final Adversarial Attack Summary on ResNet18:
FGSM:
 - Accuracy After Attack: 74.46%
 - Attack Success Rate: 25.54%
 - Time Taken: 232.66 sec

PGD:
 - Accuracy After Attack: 6.46%
 - Attack Success Rate: 93.54%
 - Time Taken: 3336.66 sec

DeepFool:
 - Accuracy After Attack: 6.04%
 - Attack Success Rate: 93.96%
 - Time Taken: 43134.19 sec

BIM:
 - Accuracy After Attack: 7.89%
 - Attack Success Rate: 92.11%
 - Time 

# **6.3 Evaluating Adversarial Robustness of EfficientNet-B0 with Adversarial Training (FGSM)**

In [14]:
# Define adversarial attacks
attacks = {
    "FGSM": torchattacks.FGSM(model_efficient, eps=0.03),
    "PGD": torchattacks.PGD(model_efficient, eps=0.03, alpha=0.01, steps=40),
    "DeepFool": torchattacks.DeepFool(model_efficient, steps=50),
    "BIM": torchattacks.BIM(model_efficient, eps=0.03, alpha=0.01, steps=10),
    "C&W": torchattacks.CW(model_efficient, c=1, kappa=0, steps=100, lr=0.01)
}

In [15]:
# Run attacks and collect results
attack_results = {}

for attack_name, attack in attacks.items():
    attack_results[attack_name] = test_adversarial_attack(model_efficient, attack, attack_name)

# Print summary
print("\nFinal Adversarial Attack Summary on EfficientNet-B0:")
for attack, (acc, success_rate, time_taken) in attack_results.items():
    print(f"{attack}:")
    print(f" - Accuracy After Attack: {acc:.2f}%")
    print(f" - Attack Success Rate: {success_rate:.2f}%")
    print(f" - Time Taken: {time_taken:.2f} sec\n")

FGSM Attack Results:
 - Accuracy after attack: 76.14%
 - Attack Success Rate: 23.86%
 - Time Taken: 416.73 seconds

PGD Attack Results:
 - Accuracy after attack: 0.49%
 - Attack Success Rate: 99.51%
 - Time Taken: 10423.11 seconds

DeepFool Attack Results:
 - Accuracy after attack: 5.60%
 - Attack Success Rate: 94.40%
 - Time Taken: 4816.68 seconds

BIM Attack Results:
 - Accuracy after attack: 3.76%
 - Attack Success Rate: 96.24%
 - Time Taken: 2663.36 seconds

C&W Attack Results:
 - Accuracy after attack: 0.91%
 - Attack Success Rate: 99.09%
 - Time Taken: 38529.86 seconds


Final Adversarial Attack Summary on EfficientNet-B0:
FGSM:
 - Accuracy After Attack: 76.14%
 - Attack Success Rate: 23.86%
 - Time Taken: 416.73 sec

PGD:
 - Accuracy After Attack: 0.49%
 - Attack Success Rate: 99.51%
 - Time Taken: 10423.11 sec

DeepFool:
 - Accuracy After Attack: 5.60%
 - Attack Success Rate: 94.40%
 - Time Taken: 4816.68 sec

BIM:
 - Accuracy After Attack: 3.76%
 - Attack Success Rate: 96.24%


## Saving Trained Models

In [16]:
torch.save(model_vgg,"model_vgg_adversarialtraining.pth")
torch.save(model_resnet,"model_resnet_adversarialtraining_.pth")
torch.save(model_efficient,"model_efficient_adversarialtraining.pth")

## Load Models if needed

In [ ]:
model_vgg=torch.load("model_vgg_adversarialtraining.pth", weights_only=False)
model_resnet=torch.load("model_resnet_adversarialtraining_.pth", weights_only=False)
model_efficient=torch.load("model_efficient_adversarialtraining.pth", weights_only=False)
model_vgg.eval()
model_resnet.eval()
model_efficient.eval()
print("Models loaded successfully")